In [17]:
import pyodbc
import json
from math import floor, ceil

In [18]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=berlinhack.database.windows.net;DATABASE=berlinhack;UID=berlinhack;PWD=Boring123')
cursor = cnxn.cursor()

Emotions file comes from our script that calls Face API on the frames of the reaction image. Topics file is the output of the Video Indexer for the content video.

In [19]:
emotions_file = 'data.json'
topics_file = 'video-indexer.json'

In [21]:
with open(emotions_file) as fp:
    emotions = json.load(fp)

with open(topics_file) as fp:
    topics = json.load(fp)

In [ ]:
video_name = 'video'
cursor.execute('INSERT INTO videos (name) OUTPUT Inserted.ID VALUES ?', video_name)
video_id = cursor.fetchone()[0]
cursor.commit()

In [26]:
result_values = []
for timestamp, face_values in emotions.items():
    for face in face_values:
        values = face['faceAttributes']
        result_values.append([video_id, int(timestamp), int(values['age']),
                      values['gender'] == 'male', values['emotion']['anger'],
                      values['emotion']['contempt'], values['emotion']['disgust'],
                      values['emotion']['fear'], values['emotion']['happiness'],
                      values['emotion']['neutral'], values['emotion']['sadness'],
                      values['emotion']['surprise']])

insights = topics['summarizedInsights']
duration = round(insights['duration']['seconds'])

In [49]:
steps = dict((s, []) for s in range(0, duration + 10, 10))
all_topics = set()

In [114]:
for keyword_item in topics['summarizedInsights']['keywords']:
    for appearance in keyword_item['appearances']:
        for s in range(round(appearance['startSeconds']) // 10 * 10,
                       (round(appearance['endSeconds']) // 10 + 1) * 10, 10):
            steps[s].append(keyword_item['name'])
            all_topics.add(keyword_item['name'])

In [89]:
topics_string = ','.join('\'' + t + '\'' for t in all_topics)
cursor.execute('''SELECT name FROM topics2 WHERE name IN ({})'''.format(topics_string))
topics_present = cursor.fetchall()[0]
to_create_topics = all_topics - set(topics_present)

In [97]:
cursor.executemany('''INSERT INTO topics2 (name) VALUES (?)''', [[i] for i in to_create_topics])
cursor.commit()

In [115]:
row_ids = []
results_query = '''
INSERT INTO results2 
(video_id, timestamp, age, male, anger,
    contempt, disgust, fear, happiness, neutral, sadness,
    surprise)
OUTPUT inserted.id
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

In [122]:
for res in result_values:
    cursor.execute(results_query, res)
    i = cursor.fetchone()
    
    stamp = res[1]
    if stamp not in steps:
        continue
        
    for topic in set(steps[stamp]):
        cursor.execute('''INSERT INTO results_topics2 (results_id, topic_id)
                       SELECT ?, id FROM topics2 WHERE name=?
                       ''', i[0], topic)
cursor.commit()